In [1]:
#load the saved matrices
import pickle
train_X=pickle.load(open( "dipep_new_train_x.p", "rb" ) )
test_X=pickle.load(open( "dipep_new_test_x.p", "rb" ) )
train_Y=pickle.load(open( "dipep_new_train_y.p", "rb" ) )
test_Y=pickle.load(open( "dipep_new_test_y.p", "rb" ) )
mean_arr=pickle.load(open( "dipep_new_mean.p", "rb" ) )
mean_std=pickle.load(open( "dipep_new_std.p", "rb" ) )

FileNotFoundError: [Errno 2] No such file or directory: 'dipep_new_train_x.p'

In [2]:
#import keras and numpy
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
#this list the devices, just making sure there is a GPU present, you might be fine with no GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 304993210606042545
]


In [4]:
#model with 2 leyers of 100 LSTM neurons
model = Sequential()
opt=Adam(lr=0.007, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.add(Dense(407, input_dim=407, kernel_initializer='random_uniform',activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200,activation='relu'))
#model.add(Dense(200,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(200,activation='relu'))
#model.add(Dense(200,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(10,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 407)               166056    
_________________________________________________________________
dropout_1 (Dropout)          (None, 407)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               81600     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2010      
Total para

In [5]:
model.fit(train_X, train_Y, epochs=200, batch_size=400,verbose=0)

In [6]:
scores = model.evaluate(test_X, test_Y, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

675/675 [==============================] - 0s 113us/step
Accuracy: 79.11%


In [7]:
from sklearn.metrics import classification_report


test_Y_index = test_Y.argmax(axis=1) # Convert one-hot to index
test_Y_predicted = model.predict_classes(test_X)
print(classification_report(test_Y_index, test_Y_predicted ))
#print (test_Y_index)

             precision    recall  f1-score   support

          0       0.89      0.79      0.84       106
          1       0.74      0.95      0.83        42
          2       0.84      0.89      0.86       109
          3       0.85      0.73      0.79        48
          4       0.91      0.86      0.89        37
          5       0.89      0.80      0.84        84
          6       0.48      0.63      0.55        76
          7       0.74      0.78      0.76        40
          8       0.45      0.34      0.39        41
          9       0.98      0.93      0.96        92

avg / total       0.80      0.79      0.79       675



In [8]:
#you can save the as a file so you dont have to train it every time
model.save('di_new_model.h5')